# Exercise F: Create an Evaluation for a prompt

In [9]:
from typing import Sequence
from aleph_alpha_client import ImagePrompt, AlephAlphaClient, AlephAlphaModel, SemanticEmbeddingRequest, SemanticRepresentation, Prompt, SummarizationRequest, CompletionRequest, EvaluationRequest
import math
import os

In [10]:
# instantiate the client and model
search_model = AlephAlphaModel(
    AlephAlphaClient(host="https://api.aleph-alpha.com", token=os.getenv("API_TOKEN")),
    model_name = "luminous-base"
)

model = AlephAlphaModel(
    AlephAlphaClient(host="https://api.aleph-alpha.com", token=os.getenv("API_TOKEN")),
    model_name = "luminous-extended"
)

In [17]:
# function for symmetric embeddings 
def embed_symmetric(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Queries
def embed_query(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Query)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Documents
def embed_document(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document)
    result = search_model.semantic_embed(request)
    return result.embedding

# function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# function for getting a summary
def generate_summary(text: str):
    request = SummarizationRequest(prompt=Prompt.from_text(text))
    result = model.summarize(request)
    return result.summary

# function that splits text by paragraphs
def split_text(text: str):
    return text.split("\n\n")

# function that evaluate two texts
def evaluate(text1: str, text2: str):
    request = EvaluationRequest(prompt=Prompt.from_text(text1), completion_expected=text2)
    result = model.evaluate(request)
    return result

In [18]:
qa_texts = [
    "WhatsApp Messenger, or simply WhatsApp, is an internationally available freeware, cross-platform centralized instant messaging (IM) and voice-over-IP (VoIP) service owned by American company Meta Platforms (formerly Facebook). It was first released in 2009.",
    "Mountain View is a city in Santa Clara County, California,[11] United States. Named for its views of the Santa Cruz Mountains,[12] it has a population of 82,376.[8]",
    "The opposite of high tech is low technology, referring to simple, often traditional or mechanical technology; for example, a slide rule is a low-tech calculating device.[5][6][7] When high tech becomes old, it becomes low tech, for example vacuum tube electronics.",
    "A startup or start-up is a company or project undertaken by an entrepreneur to seek, develop, and validate a scalable business model.[1][2] While entrepreneurship refers to all new businesses, including self-employment and businesses that never intend to become registered, startups refer to new businesses that intend to grow large beyond the solo founder.",
    ]

qa_samples = [{
    "question": "When was WhatsApp released?", 
    "correct_answer" : "2009", 
    "correct_context" : 0},
    {
    "question": "In which state is Mountain View located?", 
    "correct_answer" : "California", 
    "correct_context" : 1},
    {
    "question": "What is the opposite of high tech?", 
    "correct_answer" : "Low tech", 
    "correct_context" : 2},
    {
    "question": "Who undertakes a startup?", 
    "correct_answer" : "Entrepreneur", 
    "correct_context" : 3},
    ]

In [42]:
pre_prompt = "This system answers questions based on a text.\nText:A concept of design science was introduced in 1957 by R. Buckminster Fuller[1][2] who defined it as a systematic form of designing.\nQUestion: When was the concept of design science introduced?\nAnswer: 1957\n###\nText: The Function-Behaviour-Structure ontology – or short, the FBS ontology – is an ontology of design objects, i.e. things that have been or can be designed. The Function-Behaviour-Structure ontology conceptualizes design objects in three ontological categories.\nQuestion: How many categories are there?\nAnswer: 3\n###\n"

embeded_documents = [embed_document(text) for text in qa_texts]

def search(query: str, top_n: int = 1):
    embeded_query = embed_query(query)
    scores = [cosine_similarity(embeded_query, embeded_document) for embeded_document in embeded_documents]
    return sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]

def answer_question(question: str):
    context = search(question, top_n=1)[0]
    req = CompletionRequest(prompt=Prompt.from_text(pre_prompt + f"Context: \"{context}\"\nQuestion: \"{question}\"\nAnswer:"), stop_sequences=["###", "\n"])
    result = model.complete(req)
    return result.completions[0].completion
    

In [43]:
answer_question("What is the opposite of high tech?")

' Low tech'

### Task: 
Write an automated evaluation for both the search as well as the answering function


In [42]:
# Create a function that creates a guided summary of a text.
def guided_summary(text : str, guidance : str):
    pass

### Task 2:
Create a function that searches a list of texts based on a question and then generates an answer.
- Use search to find the most relevant text
- Use a completion request with your own prompt to generate an answer

In [25]:
texts_to_search = [
    "The judiciary of Poland (Polish: sądownictwo w Polsce) are the authorities exercising the judicial power of the Polish state on the basis of Chapter 8 of the Constitution of Poland.[a] As in almost all countries of continental Europe, the Polish judiciary operates within the framework of civil law.The courts (sądy), designated by the Constitution as those exercising the administration of justice (wymiar sprawiedliwości), are the bodies that review the vast majority of cases, with the exception of those specifically assigned to the two tribunals (trybunały). ",
    "Nicholas Blake 'Nick' Solak (born January 11, 1995) is an American professional baseball second baseman and outfielder for the Texas Rangers of Major League Baseball (MLB). Solak attended Naperville North High School in Naperville, Illinois, and the University of Louisville in Louisville, Kentucky.",
    "Die Buddhistenkrise war ein Zeitraum politisch-religiöser Anspannungen in Südvietnam vom 8. Mai bis 2. November 1963. Sie wurde durch das Verbot der buddhistischen Flagge durch die Regierung Ngô Đình Diệms ausgelöst, und endete mit einem Putsch der Armee der Republik Vietnam, wobei Ngô Đình Diệm festgenommen und später getötet wurde."
]

In [26]:
def search_and_answer(question : str):
    pass